# FAQ of Using Spellsource for AI Research

## Simulations

**Question:** Does the framework support a forward method such that an agent can receive a simulated outcome of an action?

**Answer::** Let's walk through an example of creating a game context, seeing available actions, choosing an action and looking at the game state.

First, we'll import the dependencies.

In [1]:
from spellsource.context import Context
from spellsource.behaviour import *

While normally `Context()` is best managed as a resource, it'll be convenient to have autocomplete, so we'll create a reference to it this way and close it later with `del ctx`.

In [2]:
ctx = Context()

Now we need a game context. A game context is a combination of the game data, the game logic, and a way to specify player behaviours, plus some convenience methods to play out a game.

Let's create a game context from two standard deck lists we obtained from [TempoStorm](https://tempostorm.com/hearthstone/meta-snapshot/standard/2018-06-18).

In [3]:
TOKEN_DRUID = '''### Token Druid - Standard Meta Snapshot - June 18, 2018
# Class: Druid
# Format: Standard
# Year of the Raven
#
# 2x (1) Lesser Jasper Spellstone
# 2x (2) Wild Growth
# 2x (2) Wrath
# 2x (3) Savage Roar
# 2x (4) Branching Paths
# 1x (4) Oaken Summons
# 2x (4) Soul of the Forest
# 2x (4) Swipe
# 2x (4) Violet Teacher
# 2x (4) Wispering Woods
# 2x (5) Arcane Tyrant
# 2x (5) Nourish
# 1x (6) Cairne Bloodhoof
# 2x (6) Spreading Plague
# 1x (7) Malfurion the Pestilent
# 1x (8) The Lich King
# 2x (10) Ultimate Infestation
#
AAECAZICBITmAqQDmdMCws4CDZjSAuQIxAbmBZ7SAv0CQIUI1+8C29MCX6DNAofOAgA=
#
# To use this deck, copy it to your clipboard and create a new deck in Hearthstone'''

EVEN_WARLOCK = '''### Even Warlock - Standard Meta Snapshot - June 18, 2018
# Class: Warlock
# Format: Standard
# Year of the Raven
#
# 1x (2) Acidic Swamp Ooze
# 2x (2) Defile
# 2x (2) Doomsayer
# 2x (2) Plated Beetle
# 2x (2) Sunfury Protector
# 2x (2) Vulgar Homunculus
# 2x (4) Hellfire
# 2x (4) Hooked Reaver
# 2x (4) Lesser Amethyst Spellstone
# 2x (4) Shroom Brewer
# 2x (4) Spellbreaker
# 2x (4) Twilight Drake
# 1x (6) Dread Infernal
# 1x (6) Genn Greymane
# 1x (6) Skulking Geist
# 1x (8) The Lich King
# 1x (10) Bloodreaver Gul'dan
# 2x (12) Mountain Giant
#
AAECAf0GBooH+wfN9AKgzgLCzgKX0wIM58sCigHq5gL7BvHQArYH/dACiNIC2OUC8gWNCOEHAA==
#
# To use this deck, copy it to your clipboard and create a new deck in Hearthstone
'''
game_context = ctx.GameContext.fromDeckLists([TOKEN_DRUID, EVEN_WARLOCK])

Note that the autocomplete for `ctx.GameContext` will show all the possible functions that can be called on it; the Java-Python bridge supports autocomplete.

At this particular moment, the game has not yet started. The typical lifecycle of a game looks like:

  1. Construct a `GameContext` using `fromTwoRandomDecks` or `fromDecks`.
  1. `play()`, which just calls `init(); resume();`.
     1. `init()` will call `requestMulligan` on the two `Behaviour` instances set in `game_context.getBehaviours()`.
     2. `resume` will repeatedly play turns until the end of the game, calling `requestAction` on the two `Behaviour` instances.
     
Typically, when `play` is called, the game is run until completion. However, to make it interactive in the console, we have to change the kind of agent we use. We'll use a `FiberBehaviour` to represent an interactive agent.

Let's now create instances of the two agents (the behaviours) that we'll use to actually control the game interactively.

In [4]:
agent_1 = ctx.behaviour.FiberBehaviour()
agent_2 = ctx.behaviour.FiberBehaviour()

game_context.setBehaviour(ctx.GameContext.PLAYER_1, agent_1)
game_context.setBehaviour(ctx.GameContext.PLAYER_2, agent_2)

Now, when we `play()` (i.e., `init()` and `resume()`), the game will advance until one of the agents has a pending action.

However, if we play this notebook multiple times, we'll get different results. The random seed for the game is not held constant. To do that, we'll create a `GameLogic` with a specific seed, and give it to the game context.

In [5]:
SEED = 10101
game_context.setLogic(ctx.GameLogic(SEED))

When we `play`, the game will advance until the player who starts first will mulligan. Note that in the networked version of the game, mulligans happen simultaneously.

In [6]:
game_context.play()

Let's see which agent has a mulligan pending. We'll sleep 10 ms because the game is asynchronous.

In [7]:
def pending_mulligans():
    for i, agent in enumerate([agent_1, agent_2]):
        print('Agent', i+1, [card.toString() for card in agent.getMulliganCards()])

from time import sleep
sleep(0.001)
pending_mulligans()

Agent 1 []
Agent 2 []


Observe we used `toString` to print out the description of a `Card` object.

Agent 1 has a pending mulligan request with the specified cards. Let's discard `Malfurion the Pestilent`.

In [8]:
malfurion_the_pestilent = agent_1.getMulliganCards()[2]
assert malfurion_the_pestilent.getName() == 'Malfurion the Pestilent'
agent_1.setMulligan([malfurion_the_pestilent])

Note, that to support interactive gameplay with the console, immediately printing the pending mulligans will show none, because the call to `setMulligan` is asynchronous (though near instantaneous). We'll sleep to let the simulation catch up with our command. Normally, you would not need to do this in actual implementations of `Behaviour`.

In [9]:
sleep(0.01)

Which mulligans are pending now?

In [10]:
pending_mulligans()

Agent 1 []
Agent 2 ["[MINION 'Vulgar Homunculus' [EntityReference id:45] Manacost:2]", "[MINION 'Mountain Giant' [EntityReference id:64] Manacost:12]", "[MINION 'Hooked Reaver' [EntityReference id:49] Manacost:4]", "[SPELL 'Lesser Amethyst Spellstone' [EntityReference id:52] Manacost:4]"]


Let's keep this hand for agent 2.

In [11]:
sleep(0.01)
agent_2.setMulligan([])

Which player was the active one?

In [12]:
game_context.getActivePlayer().getName()

'Player 0'

That would be agent 1. Let's get that agent's current available actions and choose one.

In [13]:
[action.toString() for action in agent_1.getValidActions()]

[]

This player can't play any cards yet. Why? Does it have enough mana?

In [14]:
[card.toString() for card in game_context.getActivePlayer().getHand()]

["[SPELL 'Lesser Jasper Spellstone' [EntityReference id:5] Manacost:1]",
 "[SPELL 'Oaken Summons' [EntityReference id:14] Manacost:4]",
 "[SPELL 'Wispering Woods' [EntityReference id:22] Manacost:4]"]

In [15]:
game_context.getActivePlayer().getMana()

1

There seems to be enough mana to play `Leser Jasper Spellstone`. But can it be played? Let's look at its text.

In [16]:
game_context.getActivePlayer().getHand()[0].getDescription()

'Deal $2 damage to a minion. (Gain 3 Armor to upgrade.)'

There are no minions on the board, so it makes sense that the card cannot be played.

What if we put a minion on the board?

In [17]:
card_bloodfen_raptor = ctx.CardCatalogue.getCardByName("Bloodfen Raptor")
assert card_bloodfen_raptor is not None

# Gets a minion from the card but which is not yet inside the game
bloodfen_raptor = card_bloodfen_raptor.summon()

# Perform the summoning effects as though the opponent of the current player summoned this minion.
opponent = game_context.getOpponent(game_context.getActivePlayer())

# int playerId, Minion minion, Card source, int index, boolean resolveBattlecry
# Should return true if the minion was successfully summoned
assert game_context.getLogic().summon(opponent.getId(), bloodfen_raptor, None, -1, False)

Jasper Spellstone now ought to be able to target this minion.

In [18]:
[action.toString() for action in agent_1.getValidActions()]

['SPELL Card: [EntityReference id:5] Target: [EntityReference id:70]',
 '[END_TURN]']

Observe that the first action specifies a card of a given entity reference and a target. Let's try resolving those references so we can see what the action is really saying.

In [19]:
for i, action in enumerate(agent_1.getValidActions()):
    source = game_context.resolveSingleTarget(action.getSourceReference())
    target = game_context.resolveSingleTarget(action.getTargetReference())
    print('%d %s %s: Targeting %s' %( i, action.getActionType().toString(), source, target))

0 SPELL [SPELL 'Lesser Jasper Spellstone' [EntityReference id:5] Manacost:1]: Targeting [MINION 'Bloodfen Raptor'id:70 3/2 hashCode: 113878665]
1 END_TURN None: Targeting None


We've now observed how to inspect some aspects of a game state and manipulate it. What if we wanted to simulate the consequences of a particular action before taking it? Let's **simulate** it by cloning the game context, executing this action and checking out the game state.

In [20]:
clone = game_context.clone()
clone_opponent = clone.getOpponent(clone.getActivePlayer())
# The clone should have a Bloodfen Raptor in the opponent's battlefield
assert clone_opponent.getMinions()[0].getSourceCard().getName() == 'Bloodfen Raptor'
# You can perform actions from one instance of a game context inside
# another; they are effectively immutable structures that don't reference
# the contexts they were created in
jasper_spellstone_action = agent_1.getValidActions()[0]
# In simulations, it's generally appropriate to just perform the game action directly instead of interacting with
# behaviours.
clone.getLogic().performGameAction(clone.getActivePlayerId(), jasper_spellstone_action)
# We should have destroyed the Bloodfen Raptor in the clone opponent's battlefield
assert len(clone_opponent.getMinions()) == 0
# The opponent from the original game context should still have the bloodfen raptor on the battlefield.
assert opponent.getMinions()[0].getSourceCard().getName() == 'Bloodfen Raptor'

Suppose we were satisfied with this simulation. Let's take this action in the original game context, and then end the turn.

In [21]:
agent_1.setAction(jasper_spellstone_action)
sleep(0.01)
assert len(agent_2.getValidActions()) == 0
end_turn_action = agent_1.getValidActions()[0]
assert end_turn_action.getActionType().toString() == 'END_TURN'
agent_1.setAction(end_turn_action)
sleep(0.01)
assert len(agent_2.getValidActions()) > 0

Let's review all the actions we did, and print a log.

In [22]:
for action in game_context.getTrace().getRawActions():
    print(action.toString())

SPELL Card: [EntityReference id:5] Target: [EntityReference id:70]
[END_TURN]


It might be helpful to see a little bit more detail about these actions. Let's resolve the source and target entities, and show which player took the action. These are not resolved by default because actions never hold references to things inside the game object, in order to let you synthesize them however you want.

In [23]:
for action in game_context.getTrace().getRawActions():
    action_type = action.getActionType().toString()
    source = action.getSource(game_context)
    player_id = source.getOwner()
    targets = action.getTargets(game_context, player_id)
    print(player_id, 
          action_type, 
          source.getName() if source is not None else "(no source)", 
          'ON' if len(targets) > 0 else '',
          ', '.join(target.getName() for target in targets))

0 SPELL Lesser Jasper Spellstone ON Bloodfen Raptor
0 END_TURN Malfurion Stormrage  


Let's close this context.

In [24]:
ctx.close()

## Speed

**Question:** How fast can games be simulated?

**Answer:** Quite fast, definitely the fastest of all Hearthstone simulators with good card coverage. However, there is significant overhead when interacting with Python, so performance-sensitive agents must necessarily be implemented in Java and use the game state as closely as possible without cloning.

Generally, other simulators perform faster with more efficient cloning. However, they typically omit or incorrectly implement effects that mutate a lot of game state, like `Prince Keleseth`. Once all card effects are implemented, simulators tend to approach the same speed.

Let's compare the Python implementation of `PlayRandomBehaviour` to the Java implementation, in order to measure the impact of the overhead of single method calls (no meaningful data transfer).

In [25]:
from tqdm import tqdm
from spellsource.utils import simulate
with Context() as ctx:
    results = list(tqdm(simulate(
        behaviours=('PlayRandomBehaviour', 'PlayRandomBehaviour'), 
        decks=(TOKEN_DRUID, EVEN_WARLOCK),
        number=1000,
        context=ctx)))

100%|██████████| 1/1 [00:09<00:00,  9.31s/it]


In [26]:
from spellsource.playrandombehaviour import PlayRandomBehaviour
with Context() as ctx:
    results = list(tqdm(simulate(
        behaviours=(PlayRandomBehaviour(), PlayRandomBehaviour()), 
        decks=(TOKEN_DRUID, EVEN_WARLOCK),
        number=1000,
        context=ctx)))

100%|██████████| 1/1 [01:04<00:00, 64.72s/it]


To simulate 1000 randomly-played games with no method overhead, it takes between 8-10s on this machine. With method overhead (the simplest possible Python behaviour), it takes about 60-80s. It's strongly recommended to implement all performance-sensitive `Behaviours` in Java.

## Stable Releases

**Question:** How does API stability look?

**Answer:** The abstract base class for `Behaviour` will never change. Until 1.0, the `GameContext` will call `requestAction` on a behaviour whenever playing a card results in a discover or battlecry action to take, resulting in possibly recursive calls to `requestAction` whenever simulating; at 1.0, there will never be recursive calls to `requestAction`.

Before 1.0:

 1. `GameContext` calls `requestAction` on a behaviour (agent) like normal. `requestAction` is pushed to the call stack.
 2. In the process of simulating all the possible actions, the agent simulates playing a Discover card.
 3. Inside the simulated `GameContext`, the discover card calls `requestAction` on the original behaviour instance. `requestAction` is pushed to the call stack a second time.
 4. The behaviour must deal with the recursive call appropriately (the current AIs do).
 5. The simulated `requestAction` is popped off the call stack.
 6. The actual `requestAction` is poppsed off the call stack.
 
After 1.0:

 1. `GameContext` calls `requestAction` on a behaviour (agent) like normal. `requestAction` is pushed to the call stack.
 2. In the process of simulating all the possible actions, the agent simulates playing a Discover card.
 3. Inside the simulated `GameContext`, the discover card enqueues the discover choices as possible actions and returns.
 4. `requestAction` is popped off the call stack.
 
## Thesis Research

Please visit [the MIT Media Lab website](https://www.media.mit.edu/publications/minionate-the-collectible-card-game/) for the thesis in MMO game design.